In [39]:
import numpy as np

In [40]:
order = ['I' + str(i) for i in range(1, 6)]
order

['I1', 'I2', 'I3', 'I4', 'I5']

In [41]:
def load_tranc(path_data, order) :
    tranction = []
    with  open(path_data, 'r') as fid:
        for lines in fid:
            str_line = list(lines.strip().split(','))
            _t = list(np.unique(str_line))
            _t.sort(key=lambda x : order.index(x))  
            tranction.append(_t)
    return tranction

In [42]:
def get_frequent(itemesets, Tranction, min_support, prev_discard):
    L = []
    support_count = []
    new_discard = []

    k = len(prev_discard)
    for i in range(len(itemesets)):
        discard_before = False
        item = itemesets[i]
        result = []
        for i in item:
            if isinstance(i, list):
                for j in i:
                    result.append(j)
            else:
                result.append(i)

        print(f"get frequent {result}")
        if k > 0:
            for it in prev_discard[k]:
                if set(it).issubset(set(result)):
                    discard_before = True
                    break

        if not discard_before:
            count = count_ocurence(result, Tranction)
            if count/len(Tranction) >= min_support:
                L.append(result)
                support_count.append(count)
            else:
                new_discard.append(result)
    return L, support_count, new_discard


In [43]:
def join_two_itemsets(it1, it2, order):
    it1.sort(key=lambda x : order.index(x))
    it2.sort(key=lambda x : order.index(x))

    for i in range(len(it1) - 1): # check befor the last one 
       if it1[i] != it2 [i] :
           return []
    
    if order.index(it1[-1]) < order.index(it2[-1]) :
        return [it1] + [it2[-1]]

    return []


In [44]:
def join_set_item(set_of_its, order):
    C = []
    for i in range(len(set_of_its)):
        for j in range(i + 1, len(set_of_its)) :
            it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
            if(len(it_out)) >0 :
                C.append(it_out)

    return C

In [45]:
# print table 
def print_table(T, support_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
        print(f'{T[k]} | {support_count[k]}')
    print("\n")

In [46]:
Tranction =  load_tranc('/home/sokhorn/sokhorn/Rupp/semester II/finalProject/apiori/appriori/youTube/data_1.txt', order)
min_support = 2/9

In [47]:
Tranction

[['I1', 'I2', 'I5'],
 ['I2', 'I4'],
 ['I2', 'I3'],
 ['I1', 'I2', 'I4'],
 ['I1', 'I3'],
 ['I2', 'I3'],
 ['I1', 'I3'],
 ['I1', 'I2', 'I3', 'I5'],
 ['I1', 'I2', 'I3']]

In [48]:
C = {}
L = {}
Discard = {}
itemset_size = 1
Discard.update({itemset_size : []})
C.update({itemset_size : [ [f ]for f in order ] })
C

{1: [['I1'], ['I2'], ['I3'], ['I4'], ['I5']]}

Create l1

In [49]:
def count_ocurence(itemset, Tranctions):
    count = 0
    for i in range(len(Tranctions)):
        if set(itemset).issubset(set(Tranction[i])):
            count += 1
    return count


In [50]:
support_count = {}
f, supp, new_discard = get_frequent(
    C[itemset_size], Tranction, min_support, Discard
)
Discard.update({itemset_size: new_discard})
L.update({itemset_size: f})
support_count.update({itemset_size: supp})
L

get frequent ['I1']
get frequent ['I2']
get frequent ['I3']
get frequent ['I4']
get frequent ['I5']


{1: [['I1'], ['I2'], ['I3'], ['I4'], ['I5']]}

C[k] (we are going to need join step from l[k - 1] )
L[k] 

In [51]:
k = itemset_size + 1
convergence = False
while not convergence:
    C.update({k: join_set_item(L[k - 1], order)})
    f, supp, new_discard = get_frequent(C[k], Tranction, min_support, Discard)
    L.update({k: f})
    Discard.update({k: new_discard})
    support_count.update({k: supp})
    if(len(L[k]) == 0):
        convergence = True
    k += 1


get frequent ['I1', 'I2']
get frequent ['I1', 'I3']
get frequent ['I1', 'I4']
get frequent ['I1', 'I5']
get frequent ['I2', 'I3']
get frequent ['I2', 'I4']
get frequent ['I2', 'I5']
get frequent ['I3', 'I4']
get frequent ['I3', 'I5']
get frequent ['I4', 'I5']
get frequent ['I1', 'I2', 'I3']
get frequent ['I1', 'I2', 'I5']
get frequent ['I1', 'I3', 'I5']
get frequent ['I2', 'I3', 'I4']
get frequent ['I2', 'I3', 'I5']
get frequent ['I2', 'I4', 'I5']
get frequent ['I1', 'I2', 'I3', 'I5']


In [52]:
num_tranction = len(Tranction)
min_conf = 0.5

**Generating th eassociation rulees**

In [53]:
from itertools import combinations, chain
def power_set(s):
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s) + 1)))

In [54]:
def write_rules(X, X_S, S, conf, supp, lift, num_tranction):
    out_rule = ""
    out_rule += f"Frequent. Itemset {X} \n"
    out_rule += f"   Rule : {list(S)} -> {list(X_S)}"
    out_rule += "   conf : {0:2.3f} ".format(conf)
    out_rule += "   supp : {0:2.3f} ".format(supp / num_tranction)
    out_rule += "   lift : {0:2.3f} ".format(lift)
    return out_rule + "\n"

In [55]:
association = ""
association_pd = {}
ass = []
for i in range(1, len(L)):
    for j in range(len(L[i])):
        s = list(power_set(set(L[i][j])))
        s.pop()
        for z in s:
            # generate the rule
            S = set(z)
            X = set(L[i][j])
            X_S = set(X).union(S)
            supp_x = count_ocurence(X, Tranction)
            supp_x_s = count_ocurence(X_S, Tranction)
            conf = supp_x / count_ocurence(S, Tranction)
            lift = conf / (supp_x_s / num_tranction)
            if(len(S) != 0):
                if conf >= min_conf and supp_x >= min_support:
                    association += write_rules(X, X_S, S,
                                               conf, supp_x,  lift, num_tranction
                                               )
                    ass.append(
                        {
                            "Cons": tuple(S),
                            "Seq": tuple(X_S),
                            "Support":  (supp_x_s / num_tranction),
                            "conf": conf,
                            "lift": lift
                        }
                    )


In [56]:
import pandas as pd

In [57]:
rules =  pd.DataFrame(ass)
rules

,Cons,Seq,Support,conf,lift
0,"(I1,)","(I1, I2)",0.444444,0.666667,1.500000
1,"(I2,)","(I1, I2)",0.444444,0.571429,1.285714
2,"(I1,)","(I1, I3)",0.444444,0.666667,1.500000
3,"(I3,)","(I1, I3)",0.444444,0.666667,1.500000
4,"(I5,)","(I1, I5)",0.222222,1.000000,4.500000
5,"(I3,)","(I3, I2)",0.444444,0.666667,1.500000
6,"(I2,)","(I3, I2)",0.444444,0.571429,1.285714
7,"(I4,)","(I4, I2)",0.222222,1.000000,4.500000
8,"(I5,)","(I2, I5)",0.222222,1.000000,4.500000
9,"(I1, I3)","(I1, I3, I2)",0.222222,0.500000,2.250000
